In [1]:
import requests
year = 2022
day = 19

url = f"https://adventofcode.com/{year}/day/{day}/input"

with open('cookie.txt', 'r') as file:
    session = file.read()

headers = {'cookie': f"session={session}"}

with open(f'day_{day}_input.txt', 'wb') as file:
    file.write(requests.get(url, headers=headers).content)
    
import os
if os.path.exists(f"day_{day}_test.txt"):
    test = [x.strip() for x in open(f'day_{day}_test.txt', 'r')]
    for line in test[:10]:
        print(line.strip())
print("---")
if os.path.exists(f"day_{day}_test_2.txt"):
    test_2 = [x.strip() for x in open(f'day_{day}_test_2.txt', 'r')]
    for line in test[:10]:
        print(line.strip())
print("---")
lines = [x.strip() for x in open(f'day_{day}_input.txt', 'r')]
for line in lines[:10]:
    print(line.strip())

Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.
---
---
Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 2 ore and 11 clay. Each geode robot costs 4 ore and 8 obsidian.
Blueprint 2: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 2 ore and 16 clay. Each geode robot costs 4 ore and 16 obsidian.
Blueprint 3: Each ore robot costs 3 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 18 clay. Each geode robot costs 2 ore and 11 obsidian.
Blueprint 4: Each ore robot costs 2 ore. Each clay robot costs 2 ore. Each obsidian robot costs 2 ore and 17 clay. Each geode robot costs 2 ore and 10 obsidian.
Blueprint 5: Each ore robot c

In [5]:
import re

run_test = True

if run_test:
    case = test
else:
    case = lines

pattern = "Blueprint (\d+): Each ore robot costs (\d+) ore. Each clay robot costs (\d+) ore. Each obsidian robot costs (\d+) ore and (\d+) clay. Each geode robot costs (\d+) ore and (\d+) obsidian."

for line in case:
    groups = re.findall(pattern,line)
    id_, ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs = [int(x) for x in groups[0]]
    print(id_, ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs)
    break
    
robots = [1,0,0,0]
resources = [0,0,0,0]

for minute in range(24):
    
    new_robots = [0,0,0,0]
    
    # try geode
    if resources[0] >= geo_ore and resources[3] >= geo_obs:
        new_robots[3] += 1
        resources[3] -= geo_obs
        resources[1] -= geo_clay
    # try obs
    elif resources[0] >= obs_ore and resources[1] >= obs_clay:
        new_robots[2] += 1
        resources[0] -= obs_ore
        resources[1] -= obs_clay
    # try clay
    elif resources[0] >= clay_ore:
        new_robots[1] += 1
        resources[0] -= clay_ore
    # try ore
    elif resources[0] >= ore_ore:
        new_robots[0] += 1
        resources[0] -= ore_ore
        
    for i in range(4):
        resources[i] += robots[i]
        
    for i in range(4):
        robots[i] += new_robots[i]
    
    print(f"minute {minute+1} : {robots}, {resources}")

#    print(robots, resources)

1 4 2 3 14 2 7
minute 1 : [1, 0, 0, 0], [1, 0, 0, 0]
minute 2 : [1, 0, 0, 0], [2, 0, 0, 0]
minute 3 : [1, 1, 0, 0], [1, 0, 0, 0]
minute 4 : [1, 1, 0, 0], [2, 1, 0, 0]
minute 5 : [1, 2, 0, 0], [1, 2, 0, 0]
minute 6 : [1, 2, 0, 0], [2, 4, 0, 0]
minute 7 : [1, 3, 0, 0], [1, 6, 0, 0]
minute 8 : [1, 3, 0, 0], [2, 9, 0, 0]
minute 9 : [1, 4, 0, 0], [1, 12, 0, 0]
minute 10 : [1, 4, 0, 0], [2, 16, 0, 0]
minute 11 : [1, 5, 0, 0], [1, 20, 0, 0]
minute 12 : [1, 5, 0, 0], [2, 25, 0, 0]
minute 13 : [1, 6, 0, 0], [1, 30, 0, 0]
minute 14 : [1, 6, 0, 0], [2, 36, 0, 0]
minute 15 : [1, 7, 0, 0], [1, 42, 0, 0]
minute 16 : [1, 7, 0, 0], [2, 49, 0, 0]
minute 17 : [1, 8, 0, 0], [1, 56, 0, 0]
minute 18 : [1, 8, 0, 0], [2, 64, 0, 0]
minute 19 : [1, 9, 0, 0], [1, 72, 0, 0]
minute 20 : [1, 9, 0, 0], [2, 81, 0, 0]
minute 21 : [1, 10, 0, 0], [1, 90, 0, 0]
minute 22 : [1, 10, 0, 0], [2, 100, 0, 0]
minute 23 : [1, 11, 0, 0], [1, 110, 0, 0]
minute 24 : [1, 11, 0, 0], [2, 121, 0, 0]


In [16]:
import re

run_test = True

if run_test:
    case = test
else:
    case = lines

pattern = "Blueprint (\d+): Each ore robot costs (\d+) ore. Each clay robot costs (\d+) ore. Each obsidian robot costs (\d+) ore and (\d+) clay. Each geode robot costs (\d+) ore and (\d+) obsidian."

for line in case:
    groups = re.findall(pattern,line)
    id_, ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs = [int(x) for x in groups[0]]
    print(id_, ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs)
    break
    

DP = {}

STOP = 18
best = 0

def advance_with_build(minute, new_robot_i, current_robots, current_resources):

    
    if minute == STOP:
        #print(f"minute {minute+1} : {current_robots}, {current_resources}")
        if current_resources[3] > 0:
            print(f"minute {minute+1} : {current_robots}, {current_resources}")
        if current_resources[3] > best:
            current_resources[3] = best
            print(f"minute {minute+1} : {current_robots}, {current_resources}")
        else:
            return current_resources
    
    resources = current_resources.copy()

    new_robots = [0,0,0,0]

    if new_robot_i == 0:
        if resources[0] >= ore_ore:
            new_robots = [1,0,0,0]
            resources[0] -= ore_ore
        else:
            return
    
    if new_robot_i == 1:
        if resources[0] >= clay_ore:
            new_robots = [0,1,0,0]
            resources[0] -= clay_ore
        else:
            return
    
    if new_robot_i == 2:
        if resources[0] >= obs_ore and resources[1] >= obs_clay:
            new_robots = [0,0,1,0]
            resources[0] -= obs_ore
            resources[1] -= obs_clay
        else:
            return
    
    if new_robot_i == 3:
        if resources[0] >= geo_ore and resources[2] >= geo_obs:
            new_robots = [0,0,0,1]
            resources[2] -= geo_obs
            resources[1] -= geo_obs
        else:
            return
                
    resources = current_resources.copy()
    robots = current_robots.copy()
    
    for i in range(4):
        resources[i] += robots[i]
        
    for i in range(4):
        robots[i] += new_robots[i]
    
    state = (minute, new_robot_i, tuple(robots))
    if state in DP:
#        print('repeated')
        #return state
        for r in range(4):
            if DP[state][r] > current_resources[r]:
                #print('better: ', state, DP[state], current_resources)
                pass
            else:
                return
    else:
        DP[state] = resources
            
    for i in reversed(range(5)):
        advance_with_build(minute + 1, i, robots, resources)
    

robots = [1,0,0,0]
resources = [0,0,0,0]
    
advance_with_build(0, 4, robots, resources)

1 4 2 3 14 2 7
minute 19 : [1, 5, 4, 1], [18, 50, 18, 1]
minute 19 : [1, 5, 4, 1], [18, 50, 18, 0]


RecursionError: maximum recursion depth exceeded in comparison

In [2]:
import re

run_test = True

if run_test:
    case = test
else:
    case = lines

pattern = "Blueprint (\d+): Each ore robot costs (\d+) ore. Each clay robot costs (\d+) ore. Each obsidian robot costs (\d+) ore and (\d+) clay. Each geode robot costs (\d+) ore and (\d+) obsidian."

for line in case:
    groups = re.findall(pattern,line)
    id_, ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs = [int(x) for x in groups[0]]
    print(id_, ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs)
    break
    

states = []
DP = {}
best = 0

def advance_with_build(minute, new_robot_i, current_robots, current_resources):

    dp_state = (minute, new_robot_i, tuple(current_robots), tuple(current_resources))
    if dp_state in DP:
#        print('yo')
        return DP[dp_state]
    else:
        DP[dp_state] = current_resources
    
    
    if minute == STOP:
        #print(f"minute {minute+1} : {current_robots}, {current_resources}")
        if current_resources[3] > 0:
            print(f"minute {minute+1} : {current_robots}, {current_resources}")
        if current_resources[3] > best:
            current_resources[3] = best
            print(f"minute {minute+1} : {current_robots}, {current_resources}")
        else:
            return current_resources
    
    resources = current_resources.copy()

    new_robots = [0,0,0,0]

    if new_robot_i == 0:
        if resources[0] >= ore_ore:
            new_robots = [1,0,0,0]
            resources[0] -= ore_ore
        else:
            return
    
    if new_robot_i == 1:
        if resources[0] >= clay_ore:
            new_robots = [0,1,0,0]
            resources[0] -= clay_ore
        else:
            return
    
    if new_robot_i == 2:
        if resources[0] >= obs_ore and resources[1] >= obs_clay:
            new_robots = [0,0,1,0]
            resources[0] -= obs_ore
            resources[1] -= obs_clay
        else:
            return
    
    if new_robot_i == 3:
        if resources[0] >= geo_ore and resources[2] >= geo_obs:
            new_robots = [0,0,0,1]
            resources[2] -= geo_obs
            resources[1] -= geo_obs
        else:
            return
                
    resources = current_resources.copy()
    robots = current_robots.copy()
    
    for i in range(4):
        resources[i] += robots[i]
        
    for i in range(4):
        robots[i] += new_robots[i]
    
    state = (minute,tuple(resources), tuple(robots))
    states.append(state)
    
    
    #print([s for s in states if s[0] == minute and not state == s])
    for s in [s for s in states if s[0] == minute and not state == s]:
        state_is_better = True
        #print([s[1][i] >= state[1][i] and s[2][i] >= state[2][i] for i in range(4)])
        if all([s[1][i] >= state[1][i] and s[2][i] >= state[2][i] for i in range(4)]):
            #print(f"state sucks: {state} {s}")
            return
    
    
    for i in reversed(range(5)):
        advance_with_build(minute + 1, i, robots, resources)
    

robots = [1,0,0,0]
resources = [0,0,0,0]
    
STOP = 12
advance_with_build(0, 4, robots, resources)

#print(states)


1 4 2 3 14 2 7


KeyboardInterrupt: 

In [61]:
import re
from collections import deque

run_test = False

if run_test:
    case = test
else:
    case = lines

pattern = "Blueprint (\d+): Each ore robot costs (\d+) ore. Each clay robot costs (\d+) ore. Each obsidian robot costs (\d+) ore and (\d+) clay. Each geode robot costs (\d+) ore and (\d+) obsidian."

ORE = 0
CLAY = 1
OBS = 2
GEO = 3


def solve(ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs):
    
    first_state = (1,0,0,0,0,0,0,0,0)
    
    highest_geodes = 0
    
    Q = deque([first_state])
    
    states = set()
    
    while Q:
        state = Q.popleft()
        #print(state)
        if state in states:
            continue
        states.add(state)
        
        current_robots = state[:4]
        current_resources = state[4:8]
        minutes = state[8]
        #print(state,current_robots, current_resources, time)
        
        if current_resources[GEO] > highest_geodes:
            print(state)
            highest_geodes = current_resources[GEO]
        
        if minutes == STOP:
            #print(state)
            continue
            
        if len(states) % 1000000 == 0:
            print(minutes,highest_geodes,len(states))
            
#         for s in [s for s in states if s[8] == minutes and not state == s]:
#             state_is_better = True
#             if all([s[i] >= state[i] for i in range(8)]):
#                 #print('state sucks')
#                 continue
        
        
        
        max_ore_required = (STOP-minutes-1)*(ore_ore + clay_ore + obs_ore + geo_ore)
        max_clay_required = (STOP-minutes-1)*(obs_clay)
        max_obs_required = (STOP-minutes-1)*(geo_obs)

        new_states = list()

        
        # make geode
        if current_resources[ORE] >= geo_ore and current_resources[OBS] >= geo_obs:
            new_resources = list(current_resources)
            new_robots = [0,0,0,1]
            new_resources[OBS] -= geo_obs
            new_resources[ORE] -= geo_ore
            new_states.append(new_robots+new_resources+[minutes+1])
        else:
            # make obs
            if current_resources[ORE] >= obs_ore and current_resources[CLAY] >= obs_clay and current_resources[OBS] < max_obs_required:
                new_resources = list(current_resources)
                new_robots = [0,0,1,0]
                new_resources[ORE] -= obs_ore
                new_resources[CLAY] -= obs_clay
                new_states.append(new_robots+new_resources+[minutes+1])
                
            # make clay
            if current_resources[ORE] >= clay_ore and current_resources[CLAY] < max_clay_required:
                new_resources = list(current_resources)
                new_robots = [0,1,0,0]
                new_resources[ORE] -= clay_ore
                new_states.append(new_robots+new_resources+[minutes+1])

            # make ore
            if current_resources[ORE] >= ore_ore and current_resources[ORE] < max_ore_required:
                new_resources = list(current_resources)
                new_robots = [1,0,0,0]
                new_resources[ORE] -= ore_ore
                new_states.append(new_robots+new_resources+[minutes+1])

            
            new_states.append([0,0,0,0]+list(current_resources)+[minutes+1])


        for new_state in new_states:
            for i in range(4):
                new_state[i+4] += current_robots[i]
                new_state[i] += current_robots[i]

            new_state[4+ORE] = min(max_ore_required,new_state[4+ORE])
            new_state[4+CLAY] = min(max_clay_required,new_state[4+CLAY])
            new_state[4+OBS] = min(max_obs_required,new_state[4+OBS])
                
        #print(len(new_states))
        #print(new_states)
        
        Q.extend([tuple(ns) for ns in new_states])
        #print(len(Q))
    return highest_geodes
            
    
STOP = 24

list_of_bests = []

for line in case:
    groups = re.findall(pattern,line)
    id_, ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs = [int(x) for x in groups[0]]
    list_of_bests.append(id_ * solve(ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs))

print(sum(list_of_bests))


(1, 3, 2, 1, 2, 15, 4, 1, 22)
(1, 3, 3, 1, 1, 7, 6, 2, 23)
(1, 3, 3, 1, 0, 0, 0, 3, 24)
(4, 11, 3, 1, 0, 0, 0, 1, 24)
19 0 1000000
(2, 10, 4, 1, 3, 23, 6, 1, 20)
20 1 2000000
(2, 10, 5, 1, 3, 16, 10, 2, 21)
22 2 3000000
(2, 10, 5, 2, 3, 26, 5, 3, 22)
(2, 10, 6, 2, 3, 17, 10, 4, 22)
(2, 10, 6, 2, 3, 17, 10, 5, 23)
(2, 10, 6, 3, 3, 17, 6, 6, 23)
(2, 10, 6, 3, 0, 0, 0, 7, 24)
(2, 10, 6, 3, 0, 0, 0, 9, 24)
(1, 3, 3, 1, 2, 12, 8, 1, 21)
(1, 3, 4, 1, 1, 5, 11, 2, 22)
22 2 1000000
(1, 3, 4, 1, 2, 8, 10, 3, 23)
(1, 3, 4, 1, 0, 0, 0, 4, 24)
(2, 6, 4, 2, 0, 0, 0, 5, 24)
18 0 1000000
19 0 2000000
(3, 7, 7, 1, 3, 16, 12, 1, 20)
20 1 3000000
(3, 7, 8, 1, 3, 15, 19, 2, 21)
21 2 4000000
(3, 7, 9, 1, 3, 14, 27, 3, 22)
(3, 7, 9, 2, 3, 8, 16, 4, 23)
(4, 7, 8, 2, 0, 0, 0, 5, 24)
(3, 7, 9, 2, 0, 0, 0, 6, 24)
23 0 1000000
(3, 5, 6, 1, 4, 9, 12, 1, 20)
20 1 1000000
(3, 5, 6, 2, 5, 14, 8, 2, 21)
(3, 5, 7, 2, 5, 12, 14, 4, 22)
(3, 5, 7, 3, 6, 6, 10, 6, 23)
(3, 5, 7, 4, 0, 0, 0, 9, 24)
20 0 1000000
(4, 6, 6, 1

In [ ]:
# Don't consider building a robot if you know the current set of robots will already provide more resources of that type
# than you'll ever need. If you've already got 90 clay by turn 20 you probably don't need to go from 10 to 11 clay robots.

STOP = 32
list_of_32_bests = []
for line in case[:3]:
    groups = re.findall(pattern,line)
    id_, ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs = [int(x) for x in groups[0]]
    list_of_32_bests.append(solve(ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs))
    
print(list_of_32_bests[0]*list_of_32_bests[1]*list_of_32_bests[2])

(1, 3, 2, 1, 2, 15, 4, 1, 22)
(1, 3, 3, 1, 1, 7, 6, 2, 23)
23 2 1000000
(1, 3, 3, 1, 2, 10, 9, 3, 24)
24 3 2000000
24 3 3000000
(1, 3, 3, 1, 3, 13, 12, 4, 25)
25 4 4000000
25 4 5000000
25 4 6000000
(1, 3, 4, 1, 2, 5, 15, 5, 26)
(1, 3, 2, 2, 2, 27, 4, 6, 26)
26 6 7000000
